# Recap 
First, we split the data into training and test.

In [1]:
import pandas as pd

df = pd.read_csv("https://dlsun.github.io/pods/data/bordeaux.csv",
                 index_col="year")
df_train = df.loc[:1980].copy()
df_test = df.loc[1981:].copy()

In [2]:
X_train = df_train[["win", "summer"]]
y_train = df_train["price"]

X_test = df_test[["win", "summer"]]

Next, we fit a 5-nearest neighbors model to the training data and use the model to predict the labels on the test data.

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=5))
pipeline.fit(X=X_train, y=y_train)
pipeline.predict(X=X_test)

array([35.8, 54. , 52.2, 18.4, 35.6, 13.2, 37. , 51.4, 36.6, 36.6, 40.6])

# Training Error
On the training data, we know the true labels  y1,y2,...,yn , so we can calculate the training error of our predictions  y^1,y^2,...,y^n :


        $$ \text{training MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat y_i)^2 $$

In [4]:
pipeline.fit(X_train, y_train)
y_train_ = pipeline.predict(X_train)
((y_train - y_train_) ** 2).mean()

np.float64(207.24148148148146)

There's also a Scikit-Learn function for that!

In [5]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, y_train_)

207.24148148148146

To see the problem with training error, let's calculate the training error of a  1 -nearest neighbor model.

In [7]:
pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=1))
pipeline.fit(X=X_train, y=y_train)
y_train_ = pipeline.predict(X=X_train)
mean_squared_error(y_train, y_train_)

0.0

A 1-nearest neighbors model will always be perfect on the training data!

# Estimating Test Error
First, we choose half of the vintages in the training data to be in the training set. The remaining vintages will be in the validation set.

We can set a random seed (https://en.wikipedia.org/wiki/Random_seed) to ensure reproducible results.

In [10]:
import numpy as np

np.random.seed(0) # fix the randomness so that the result is the same every time it runs.
inds_train_set = np.random.choice(df_train.index, size=len(df_train) // 2,
                                  replace=False) # Selects halves randomly without replacement (replace=False).
inds_train_set

array([1955, 1978, 1968, 1971, 1959, 1965, 1977, 1967, 1973, 1974, 1970,
       1953, 1964])

In [11]:
#Eğitim verisi → ikiye bölünüyor: train subset ve validation subset.
df_train_set = df_train.loc[inds_train_set] #Rastgele seçilen indekslerden oluşan eğitim alt seti
df_val_set = df_train.drop(inds_train_set, axis="rows") #Eğitim alt seti dışındaki satırlardan oluşan doğrulama (validation) seti

Notice that since both the training and validation sets came from the training data, both the inputs  X  and the labels  y  are known.

In [12]:
X_train_set, y_train_set = df_train_set[["win", "summer"]], df_train_set["price"]
X_val_set, y_val_set = df_val_set[["win", "summer"]], df_val_set["price"] 

Now, let's fit a 1-nearest neighbors model to the training set and calculate the MSE of the predictions on the validation set.

In [13]:
pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=1))
pipeline.fit(X_train_set, y_train_set)
y_val_set_ = pipeline.predict(X_val_set)
mean_squared_error(y_val_set, y_val_set_)

195.71428571428572

Notice that the validation MSE is no longer 0!

# Cross-Validation
Previously, we fit the model to the training set and evaluated the predictions on the validation set.

In [14]:
pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=1))
pipeline.fit(X_train_set, y_train_set)
y_val_set_ = pipeline.predict(X_val_set)
mean_squared_error(y_val_set, y_val_set_)

195.71428571428572

Now let's do the same thing, with the roles of the training and validation sets reversed.

In [15]:
pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=1))
pipeline.fit(X_val_set, y_val_set)
y_train_set_ = pipeline.predict(X_train_set)
mean_squared_error(y_train_set, y_train_set_)

306.9230769230769

We get two very different answers! To come up with one overall estimate of test error, we can average them.

In [16]:
(195.71 + 306.92) / 2

251.315

We can let Scikit-Learn handle cross-validation, using the cross_val_score function.

In [17]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
    pipeline,
    X=df_train[["win", "summer"]],
    y=df_train["price"],         # this is all of the training data!
    scoring="neg_mean_squared_error", # higher is better for a score
    cv=4)
scores

array([-547.        , -405.85714286,  -67.        ,  -31.        ])

In [18]:
# To get the MSE, we need to negate the negative MSE!
-scores.mean()
#RMSE ≈ 16.21 modelin tahmin ettiği şarap fiyatlarının gerçek fiyatlardan ortalama olarak yaklaşık 16.2 birim saptığı anlamına gelir.

np.float64(262.7142857142857)